In [156]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

targetImage = cv2.imread('images/zr.jpeg', cv2.IMREAD_COLOR)

In [157]:
def print_image(image):
    cv2.namedWindow("Sample Image", cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Sample Image', 800, 800)
    cv2.imshow('Sample Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [158]:
# print_image(targetImage)

In [159]:
# print(targetImage.shape)
shapeY = targetImage.shape[0]
shapeX = targetImage.shape[1]

diff = 40

In [160]:
#윤곽선 구하는 코드
boxCount = 10



outlines = []
boxRangeX = shapeX/boxCount
boxRangeY = shapeY/boxCount
outlineBox = np.zeros((boxCount, boxCount, 0)).tolist()
outlineOnlyImage = np.zeros((shapeY,shapeX,3), dtype=np.uint8)
outlinedImage = targetImage.copy()
# outlineImage = targetImage.copy()

for y in range(0, shapeY-1):
    for x in range(0, shapeX-1):
        if(abs(np.linalg.norm(cv2.add(targetImage[y][x], -1*targetImage[y+1][x]))) >= diff or 
           abs(np.linalg.norm(cv2.add(targetImage[y][x], -1*targetImage[y][x+1]))) >= diff):
            outlines.append([y,x])
            outlineOnlyImage[y][x] = [255,0,0]
            outlinedImage[y][x] = [0,0,0]
            outlineBox[int(y//boxRangeY)][int(x//boxRangeX)].append([y,x])

In [161]:
#부분별 윤곽선 체크 코드
def checkBoxImages(boxX,boxY):
    newImage = np.zeros((shapeY, shapeX, 3), dtype=np.uint8)
    print(len(outlineBox[boxY][boxX]))
    for outlinePoint in outlineBox[boxY][boxX]:
        newImage[outlinePoint[0]][outlinePoint[1]] = [255,0,0]
    
    print_image(newImage)

In [162]:
# print_image(outlineOnlyImage)

In [163]:
# print_image(outlinedImage)

In [164]:
#박스 체크해가면서 가장 가까운 윤곽선과의 거리 구하기

boxCheck = []

def getMinDistanceStart(point):
    boxCheck.clear()
    dstP = getMinDistance(np.array(point), int(point[0]//boxRangeY), int(point[1]//boxRangeX), np.linalg.norm(np.array([shapeY,shapeX])))

    return dstP

def getMinDistance(point, boxY, boxX, minDstP):
    if([boxY, boxX] in boxCheck):
        return minDstP
    else:
        boxCheck.append([boxY, boxX])

    minDst = minDstP
    for outlinePoint in outlineBox[boxY][boxX]:
        dst = np.linalg.norm(np.array(point) - np.array(outlinePoint))
        if(dst < minDst):
            minDst = dst
            

    if(abs(boxRangeX*boxX - point[1]) <= minDst and boxX > 0):
        newDst = getMinDistance(point, boxY, boxX-1, minDst)
        if(newDst < minDst):
            minDst = newDst
    if(abs(boxRangeX*(boxX+1) - point[1]) <= minDst and boxX < boxCount - 1):
        newDst = getMinDistance(point, boxY, boxX+1, minDst)
        if(newDst < minDst):
            minDst = newDst
    if(abs(boxRangeY*boxY - point[0]) <= minDst and boxY > 0):
        newDst = getMinDistance(point, boxY-1, boxX, minDst)
        if(newDst < minDst):
            minDst = newDst
    if(abs(boxRangeY*(boxY+1) - point[0]) <= minDst and boxY < boxCount - 1):
        newDst = getMinDistance(point, boxY+1, boxX, minDst)
        if(newDst < minDst):
            minDst = newDst

    if(minDst < 1):
        return 1

    return minDst

In [165]:
#우선순위 이미지 만들기 -> priorityImage

priorityImage = np.zeros((shapeY,shapeX,3), dtype=np.uint8)

for y in range(0,shapeY):
    for x in range(0, shapeX):
        dstP = getMinDistanceStart([y,x])
        priorityImage[y][x] = [int(255//(dstP)), int(255//(dstP)), int(255//(dstP))]

In [166]:
# print_image(priorityImage)

In [167]:
#팔레트 프린트

def print_palette(colors):
    palette = np.zeros((100, 1500, 3), dtype=np.uint8)
    for i in range(len(colors)):
        for y in range(100):
            for x in range(int(1500//len(colors))):
                palette[y][i*int(1500//len(colors)) + x] = colors[i]
    
    print_image(palette)

In [168]:
#diff를 통해 palette, paletteColors, paletteColorsPixels 구하기
#palette - 평균 색상
#paletteColors - 해당 팔레트 색상 속에 포함된 모든 색상
#paletteColorsPixels - 해당 팔레트 색상 속에 포함된 모든 픽셀

def getPalette(image, diffC):
    palette = []
    paletteColors = []
    paletteColorsPixels = []

    def getNearestColor(color, pixel):
        for i, pColor in enumerate(palette):
            newDst = abs(np.linalg.norm(cv2.add(np.array(color) ,-1*np.array(pColor))))
            if(newDst <= diffC):
                paletteColors[i].append(np.array(color))
                paletteColorsPixels[i].append(pixel)
                # print(cv2.add(np.array(palette[i])*(len(paletteColors[i])-1), np.array(color))/len(paletteColors[i]))
                # palette[i] = np.array(paletteColors[i]).mean(axis = 0)
                palette[i] = cv2.add(np.array(palette[i])*(len(paletteColors[i])-1), np.array(color))/len(paletteColors[i])
                return i
        
        palette.append(color)
        paletteColors.append([color])
        paletteColorsPixels.append([pixel])
        return len(palette)-1

    simpleIndexImage = np.zeros((shapeY,shapeX))

    for y in range(0, shapeY):
        for x in range(0, shapeX):
            simpleIndexImage[y][x] = getNearestColor(image[y][x], [y,x])


    for i in range(len(palette)):
        color = palette[i]
        palette[i] = [int(color[0]), int(color[1]), int(color[2])]
    
    return palette, simpleIndexImage, paletteColorsPixels

In [169]:
#타겟 이미지 팔레트 만들고, 팔레트 대표 색상들로만 이루어진 simpleTargetImage만들기

targetPalette, simpleIndexTargetImage, targetPaletteColorsPixels = getPalette(outlinedImage, diff)

simpleTargetImage = np.zeros((shapeY,shapeX,3), dtype=np.uint8)

for y in range(0, shapeY):
    for x in range(0, shapeX):
        simpleTargetImage[y][x] = targetPalette[int(simpleIndexTargetImage[y][x])]

In [170]:
# print(len(targetPalette))
# print_image(simpleTargetImage)
# print_palette(targetPalette)

In [171]:
#랜덤 컬러 시트 만들기
colorSheetImage = np.zeros((shapeY,shapeX,3), dtype=np.uint8)

for y in range(0,shapeY):
    for x in range(0, shapeX):
        colorSheetImage[y][x] = [np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255)]

In [172]:
# print_image(colorSheetImage)

In [173]:
#goalColorCount +- goalRange 길이의 컬러 시트 팔레트를 만들기 위한 적절한 diff값을 찾는 코드

goalColorCount = len(targetPalette)
currentDiff = 0
upperDiff = 442
lowerDiff = 0
goalRange = 5

colorSheetPalette = []
colorSheetPaletteColorsPixels = []
colorSheetDiff = 0

while(True):
    currentDiff = (lowerDiff + upperDiff)/2
    newColorPalette, _, _ = getPalette(colorSheetImage, currentDiff)
    colorCount = len(newColorPalette)

    if(abs(colorCount - goalColorCount) <= goalRange):
        colorSheetDiff = currentDiff
        print("Done!!")
        print("goalColorCount: " + str(goalColorCount) + "    ColorCount: " + str(colorCount))
        print("Diff: " + str(colorSheetDiff))
        print()

        colorSheetPalette, _, colorSheetPaletteColorsPixels = getPalette(colorSheetImage, currentDiff)
        break

    elif(colorCount > goalColorCount):
        lowerDiff = currentDiff
    
    elif(colorCount < goalColorCount):
        upperDiff = currentDiff
    

    print("Diff: " + str(currentDiff) + "    ColorCount: " + str(colorCount))
    print("upperDiff: " + str(upperDiff) + "    lowerDiff: " + str(lowerDiff))
    print()
    

Diff: 221.0    ColorCount: 1
upperDiff: 221.0    lowerDiff: 0

Done!!
goalColorCount: 13    ColorCount: 15
Diff: 110.5



In [174]:
#팔레트 구분이 잘 되었나 확인하는 코드
def print_colorSheetPaletteColorsPixels(colorIdx):
    colorImage = np.zeros((shapeY, shapeX, 3), dtype=np.uint8)

    for i in range(len(colorSheetPaletteColorsPixels[colorIdx])):
        position = colorSheetPaletteColorsPixels[colorIdx][i]
        colorImage[position[0]][position[1]] = colorSheetImage[position[0]][position[1]]

    print_image(colorImage)

In [175]:
# print_colorSheetPaletteColorsPixels(3)
# print_palette(colorSheetPalette)

In [176]:
# 팔레트를 가장 많이 사용된 순서로 정렬하는 함수

def sortPaletteByVolume(paletteO, paletteColorsPixelsO):
    palette = paletteO.copy()
    paletteColorsPixels = paletteColorsPixelsO.copy()
    for i in range(len(palette) - 1):
        for j in range(i+1, len(palette)):
            if(len(paletteColorsPixels[i]) < len(paletteColorsPixels[j])):
                c = palette[i].copy()
                palette[i] = palette[j].copy()
                palette[j] = c.copy()

                l = paletteColorsPixels[i].copy()
                paletteColorsPixels[i] = paletteColorsPixels[j].copy()
                paletteColorsPixels[j] = l.copy()

    return palette, paletteColorsPixels

In [177]:
# 타겟팔레트 정렬 후 타겟팔레트의 각 색상의 픽셀 개수를 담은 배열 생성

targetPalette, targetPaletteColorsPixels = sortPaletteByVolume(targetPalette, targetPaletteColorsPixels)

targetPaletteColorCount = [len(targetPaletteColorsPixels[i]) for i in range(len(targetPaletteColorsPixels))]

In [178]:
# print_palette(targetPalette)
# print(targetPaletteColorCount)

In [179]:
# 컬러시트팔레트 정렬 후 컬러시트팔레트의 각 색상의 픽셀 개수를 담은 배열 생성

colorSheetPalette, colorSheetPaletteColorsPixels = sortPaletteByVolume(colorSheetPalette, colorSheetPaletteColorsPixels)

colorSheetPaletteColorCount = [len(colorSheetPaletteColorsPixels[i]) for i in range(len(colorSheetPaletteColorsPixels))]

In [180]:
# print_palette(colorSheetPalette)
# print(colorSheetPaletteColorCount)

각 팔레트 색상의 픽셀 개수에 따라서
배합하는 시트색상의 수를 1개부터 차례로 늘려가며 타겟색상을 만든다
가장 적은 배합색상으로, 타겟 팔레트에서 가장 많이 사용되는 색상부터 시작하여
각 색상을 만들 때마다 컬러시트팔레트에서 가장 많이 사용되는 색상들을 우선적으로 사용하여
타겟 색상 팔레트의 색상 하나당 컬러시트의 색상이 몇%의 비율로 들어가야하는지 저장하는 배열을 생성한다
또한 소숫점 오차를 없애기 위해 그저 픽셀수 자체로 동작하게 만들었다.
colorProportionsPixelCount에서 정확한 각 색상의 픽셀수를 확인할 수 있다

+
각 색상의 배합 색상 조합은 고유하다. (비율에 상관 없이)



!
따라서 컬러시트 색상이 너무 부족할 경우 제대로 동작하지 않을 수 있다.
예시로 컬러시트 색상의 모든 조합의 개수보다 타겟컬러 색상의 개수가 클 경우에는 동작하지 않는다.

In [181]:

#나머지 부분은 막 속도가 폐기물급은 아니다. 근데 저기 sort부분이 진짜 개더럽게 오래걸린다. 탐색부분이 문제가 아니라 지금 정렬이 너무 오래걸리는게 문제다.
#근데 어차피 조합되는 순서가 볼륨을 따라가고, 조합되는 색상의 개수가 일정한데 꼭 정렬을 해야하는지부터 생각해봐야겠다. -> 100 3 3 과 50 40 30 이런 경우가 있을 수 있으니 정렬 무의미x
#근데 어차피 배열되는 색상 수는 같은데 그냥 정렬을 안 해도 그렇게 이상하지는 않을 수도?
#예를 들어 더 큰 색상을 만들 수 있는 조합으로 적은걸 만들어버릴 수는 있지만, 이정도는 성능과 타협하는 부분으로 치자. 정 싫으면 나중에 정렬만 다시 짜던지

from itertools import *

resultColorCount = 0
indices = np.arange(len(colorSheetPalette))
mergeColorCount = 1
colorProportions = np.zeros((len(targetPalette), len(colorSheetPalette)))
colorProportionsPixelCount = np.zeros((len(targetPalette), len(colorSheetPalette)))

colorSheetPaletteColorCountUsable = colorSheetPaletteColorCount.copy()
doneTargetColors = np.zeros((len(targetPalette)))

while(resultColorCount < len(targetPalette) and mergeColorCount <= len(colorSheetPalette)):
    # print("combination")
    mergeColors = list(combinations(indices, mergeColorCount))
    # print("combination done")

    # combinationColorCount = np.zeros(len(mergeColors))

    # # print("sum and check")
    # for i in range(len(mergeColors)):
    #     for j in range(len(mergeColors[i])):
    #         if(colorSheetPaletteColorCountUsable[mergeColors[i][j]] <= 0):
    #             combinationColorCount[i] = 0
    #             break
    #         combinationColorCount[i] += colorSheetPaletteColorCountUsable[mergeColors[i][j]]

    # # print("sort")
    # mergeColors = sorted(mergeColors, key = lambda x : combinationColorCount[mergeColors.index(x)], reverse=True)

    # print("start check")
    for i in range(len(mergeColors)):
        cnt = 0

        colors = mergeColors[i]

        doCheck = True

        # print("getcnt")
        for j in colors:
            if(colorSheetPaletteColorCountUsable[j] <= 0): doCheck = False
            else : cnt += colorSheetPaletteColorCountUsable[j]
        
        if(doCheck == False):
            continue
        # print("getIdx")

        colorMakeIdx = -1
        for k in range(len(targetPalette)):
            if(cnt >= targetPaletteColorCount[len(targetPalette) - 1 - k]):
                if(doneTargetColors[len(targetPalette) - 1 - k] != 1):
                    colorMakeIdx = len(targetPalette) - 1 - k
                else:
                    pass
            else:
                break
        
        # print("makeColor")
        if(colorMakeIdx != -1):
            colors = sorted(mergeColors[i], key = lambda x : colorSheetPaletteColorCountUsable[x])
            cnt = 0
            amount = -1
            amountPixel = -1

            for j in colors:
                doneTargetColors[colorMakeIdx] = 1

                if(targetPaletteColorCount[colorMakeIdx] - cnt > colorSheetPaletteColorCountUsable[j]):
                    colorProportions[colorMakeIdx][j] = round(colorSheetPaletteColorCountUsable[j] / targetPaletteColorCount[colorMakeIdx], 3)
                    colorProportionsPixelCount[colorMakeIdx][j] = colorSheetPaletteColorCountUsable[j]
                    cnt += colorSheetPaletteColorCountUsable[j]
                    colorSheetPaletteColorCountUsable[j] = 0
                else:
                    if((colors.index(j)) == len(colors) - 1):
                        colorProportionsPixelCount[colorMakeIdx][j] = targetPaletteColorCount[colorMakeIdx] - cnt
                        colorProportions[colorMakeIdx][j] = round(targetPaletteColorCount[colorMakeIdx] - cnt / targetPaletteColorCount[colorMakeIdx], 3)
                        colorSheetPaletteColorCountUsable[j] -= targetPaletteColorCount[colorMakeIdx] - cnt
                        
                    else:
                        if(amount < 0): 
                            amount = 1//(len(colors) - (colors.index(j))) * (targetPaletteColorCount[colorMakeIdx] - cnt)
                            amountPixel = amount
                            
                        colorProportions[colorMakeIdx][j] = round(amount / targetPaletteColorCount[colorMakeIdx], 3)
                        colorProportionsPixelCount[colorMakeIdx][j] = amount
                        cnt += amount
                        colorSheetPaletteColorCountUsable[j] -= amount
                
                if(colorSheetPaletteColorCountUsable[j] <= 0):
                    indices = np.delete(indices, list(indices).index(j))

            resultColorCount += 1
    
    print(mergeColorCount)
    mergeColorCount += 1
    
if(resultColorCount < len(targetPalette)):
    print("failed")

1
2
3
4
5
6
7
8
9
10
11
12
13


In [182]:
print(doneTargetColors)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [183]:
# 다 만든 후 남은 컬러시트 픽셀 개수
print(sum(colorSheetPaletteColorCountUsable))

0


In [184]:
# cnt = 0
# for i in range(len(colorProportionsPixelCount)):
#     for j in range(len(colorProportionsPixelCount[i])):
#         cnt += colorProportionsPixelCount[i][j]

# print(cnt)

In [185]:
# 각 색상의 비율을 random하게 뽑을 수 있게 0~1사이의 범위를 나타내게 만들고 ,각 색상의 비율에 따라 사용할 컬러시트 픽셀의 개수를 지정한다

colorProportionRanges = colorProportions.copy()
colorProportionSecuredCount = colorProportions.copy() 
# 저장된 개수는 소숫점의 반올림 등으로 인해 실제 개수보다 1개 이상 차이날 수 있다. 그러니 다 썼으면 그냥 랜덤치는 예외케이스 구현 필요

for i in range(len(colorProportions)):
    cnt = 0
    for j in range(len(colorProportions[i])):
        if(colorProportions[i][j] == 0):
            colorProportionRanges[i][j] = 0
            colorProportionSecuredCount[i][j] = 0
        else:
            cnt += colorProportions[i][j]
            colorProportionRanges[i][j] = cnt
            colorProportionSecuredCount[i][j] = int(len(targetPaletteColorsPixels[i]) * colorProportions[i][j])

In [186]:
# print(colorProportionRanges)
# print(colorProportionSecuredCount)

In [187]:
#팔레트 색상에 가장 일치하는 픽셀로 정렬하기 (윤곽선 가까이에 우선적으로 사용할 것)
for i in range(len(colorSheetPalette)):
    colorSheetPaletteColorsPixels[i] = sorted(colorSheetPaletteColorsPixels[i], key = lambda x : (abs(np.linalg.norm(cv2.add(np.array(colorSheetPalette[i]), -1*np.array(x))))))

In [188]:
#윤곽선과 가까운 정도(우선도)에 따라 step을 나눠준다.

stepPixels = {}
steps = []

for y in range(shapeY):
    for x in range(shapeX):
        if(str(priorityImage[y][x][0]) not in stepPixels):
            steps.append(str(priorityImage[y][x][0]))
            stepPixels[str(priorityImage[y][x][0])] = [[y,x]]
        else:
            stepPixels[str(priorityImage[y][x][0])].append([y,x])

steps = sorted(steps, key = lambda x : int(x), reverse=True)

In [189]:
# print(steps)
# print(stepPixels)

In [190]:
def getIndex(l, a):
    for i in range(len(l)):
        if(all(l[i] == a)):
            return i
    
    return 0

In [191]:
# 위에서 구한 색 배합 비율에 따라 랜덤으로 어떤 색상을 사용할지 return한다. 만약 할당된 모든 색상의 픽셀을 다 썼을 경우 -1을 return한다.
# 실제 필요한 픽셀 수와 할당된 픽셀 수는 거의 같으므로 아주 적은 픽셀들에만 -1이 return된다

def getColor(proportionRange):

    colorIdx = -1

    rand = np.random.rand(1)

    for i in range(len(proportionRange)):
        if((rand <= proportionRange[i] and proportionRange[i] != 0)):
            colorIdx = i
            break

    return colorIdx


In [192]:
def getProportionRange(proportionPixelCount):
    proportionPixelCountRange = proportionPixelCount.copy()
    cnt = 0
    for i in range(len(proportionPixelCount)):
        if(proportionPixelCount[i] != 0):
            cnt += proportionPixelCount[i]
            proportionPixelCountRange[i] = cnt

    return np.array(proportionPixelCountRange) / cnt

step(우선도) 에 따라 윤곽선에 가까운 픽셀부터 채워준다.
팔레트가 표현하는 색상에 많이 일치하는 색상부터 우선적으로 윤곽선을 표현하는데에 사용하기 위해서이다.

오차를 줄이고, 다 사용한 컬러 제외처리 등을 위해 앞서 구했던 proportion이 아닌, 사용 가능한 컬러 픽셀의 개수에 따라 매 번 getProportionRange를 하게 만들었다

In [ ]:
cntNull = 0 #예외케이스를 잡기 위한 카운트

resultImage = np.zeros((shapeY,shapeX,3), dtype=np.uint8)
resultPosImage = np.zeros((shapeY, shapeX, 2))

colorProportionsPixelCountCopy = colorProportionsPixelCount.copy()
colorSheetPaletteColorsPixelsIdx = np.zeros((len(colorSheetPalette)), dtype=np.int64)

for step in steps:
    for y,x in stepPixels[step]:
        pixelColor = simpleTargetImage[y][x]
        pixelColorIdx = getIndex(targetPalette, pixelColor)

        pixelColorProportionRange = getProportionRange(colorProportionsPixelCountCopy[pixelColorIdx])
        
        newPixelColorIdx = getColor(pixelColorProportionRange)

        if(newPixelColorIdx != -1):
            
            pixelPosition = colorSheetPaletteColorsPixels[newPixelColorIdx][colorSheetPaletteColorsPixelsIdx[newPixelColorIdx]]
            newPixelColor = colorSheetImage[pixelPosition[0]][pixelPosition[1]]
            resultImage[y][x] = newPixelColor
            resultPosImage[pixelPosition[0]][pixelPosition[1]] = [y,x]


            colorSheetPaletteColorsPixelsIdx[newPixelColorIdx] += 1
            colorProportionsPixelCountCopy[pixelColorIdx][newPixelColorIdx] -= 1

        else:
            cntNull += 1
            resultImage[y][x] = [255,0,0]



#이거 다른 타겟 색상 만들 때 다 쓰면 그 색상 비율만 바꾸고 다른 그 색상을 포함하는 색상 비율은 안바꿔서 문제됨.
#그냥 색상 딱 체크 할 때 다시 그거 체크하는걸로 하는게 나을 것 같기도 한데

In [197]:
print_image(resultImage)

In [195]:
print(cntNull)

0


이제 resize랑 diff 설정하는거 만들고
그 다음 이제 컬러팔레트 팔레트 개수 지정 - 노이즈 기준
컬러팔레트 이미지로 지정 - img 2 img
하고 그 다음 애니메이션 pygame써서 하자
그러면 시작좌표 끝좌표를 해야할 것 같고, 이쯤에서 한 번 파일 나눠줄 필요가 있을듯

그리고 저기 색상 만드는거랑 색상 칠하는거 리펙토링 주석 다시 달고
색상 만드는 거 구현 좀 다르게 더 퀄리티 높일 수는 없나?
지금의 아이디어는 최소한의 색의 배합부터(정렬때문에 조금 어긋나기는 했지만) 하는거고
가장 서로 다른 색상을 만드는건 또 좀 다른 얘기이긴 한데
결국에 컬러시트에도 diff가 있고, 서로 다른 색끼리 배합을 하는거면 겹칠 일이 적을 것 같기는 한데 그냥 이정도에서 만족을 해버릴까?

컬러별로 나눠진 각 픽셀들의 원 좌표, 목표 좌표를 가지는 배열이 필요하고
시간에 따라서 현 좌표를 원 좌표와 목표 좌표의 내분점으로 만들어야겠다.
그건 numpy array를 사용한 연산으로 구현 가능할거고
원래 사용하던 colorpixels배열 사용할 수 있겠다.

생각해보니까 plt는 결국 좌표에 따라 plot하는거고... 그걸 확대, 축소도 가능하니까 맨 처음에 의도했던 여러 사진 구현도 완벽히 가능하겠는데?
결국 벡터니까 뭉게질 걱정을 할 필요가 없네


근데 그러면 이건 어떄?
모든 컬러(뭉게진거 말고)에 대해서
[[oy,ox], [ty,tx]]를 가지는...

In [ ]:
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(8,8))
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
matplotlib.rcParams['animation.embed_limit'] = 2**128
matplotlib.rcParams['animation.ffmpeg_path'] = r'/opt/homebrew/Cellar/ffmpeg/8.0.1/bin/ffmpeg'
plt.ioff()
fig, ax = plt.subplots()


pltXoriginal = []
pltYoriginal = []
pltXtarget = []
pltYtarget = []
pltColor = []


for y in range(shapeY):
    for x in range(shapeX):
        pltXoriginal.append(x)
        pltYoriginal.append(shapeY - y)
        pltColor.append(np.array(colorSheetImage[y][x][::-1])/255)
        targetPos = resultPosImage[y][x]
        pltXtarget.append(targetPos[1])
        pltYtarget.append(shapeY - targetPos[0])

beforeChangeFrame = 30
changeFrame = 180
afterChangeFrame = 30
pltXoriginal = np.array(pltXoriginal)
pltYoriginal = np.array(pltYoriginal)
pltXtarget = np.array(pltXtarget)
pltYtarget = np.array(pltYtarget)

def animate(t):
    f = changeFrame - beforeChangeFrame
    if(t - beforeChangeFrame < 0 or t > beforeChangeFrame + changeFrame):
        pass
    else:
        fx = ((changeFrame - (t - beforeChangeFrame))*pltXoriginal + ((t - beforeChangeFrame))*pltXtarget)/changeFrame
        fy = ((changeFrame - (t - beforeChangeFrame))*pltYoriginal + ((t - beforeChangeFrame))*pltYtarget)/changeFrame
        plt.cla()
    
        plt.xlim(0,shapeX)
        plt.ylim(0,shapeY)
        plt.scatter(fx, fy, c = pltColor, marker='s', s = 2**2)

plt.cla() 
plt.xlim(0,shapeX)
plt.ylim(0,shapeY)
plt.scatter(pltXoriginal, pltYoriginal, c = pltColor, marker='s', s = 2**2)
anim = matplotlib.animation.FuncAnimation(fig, animate, frames=beforeChangeFrame + changeFrame + afterChangeFrame, interval=1000/60, repeat = False)


writer = matplotlib.animation.FFMpegFileWriter(fps = 60)
anim.save("videos/" + "a_to_b" +".mp4", writer='ffmpeg')